In [16]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

import plotly.graph_objects as go

In [17]:
sankey_df = pd.read_csv('sankeyDiagram.csv', header = None)
sankey_df.columns = ['start', 'end', 'New IP', 'User Type']

In [18]:
sankey_df

,start,end,New IP,User Type
0,뉴스,매거진,1000201,내부
1,채용/인재,검색,1000207,내부
2,검색,기타,1000207,내부
3,기타,검색,1000207,내부
4,검색,기타,1000207,내부
...,...,...,...,...
411636,기타,고객센터,999994,외부
411637,고객센터,기타,999994,외부
411638,기타,고객센터,999994,외부
411639,고객센터,기타,999994,외부


In [19]:
#'이탈 경로' 추가
leave_df = sankey_df.drop_duplicates(subset=['New IP'], keep='last')
leave_df = leave_df.drop('start', axis=1)
leave_df = leave_df.rename(columns={'end': 'start'})
leave_df['end'] = '이탈'
leave_df

,start,New IP,User Type,end
0,매거진,1000201,내부,이탈
7,채용/인재,1000207,내부,이탈
17,검색,1000216,내부,이탈
19,매거진,1000226,내부,이탈
56,뉴스,1000236,내부,이탈
...,...,...,...,...
411626,기타,999960,외부,이탈
411627,고객센터,999973,외부,이탈
411631,기타,999980,외부,이탈
411639,기타,999994,외부,이탈


In [20]:
leave_df.to_csv('leave_df.csv', index=False)

In [21]:
combined = pd.concat([sankey_df, leave_df], axis=0)
combined

,start,end,New IP,User Type
0,뉴스,매거진,1000201,내부
1,채용/인재,검색,1000207,내부
2,검색,기타,1000207,내부
3,기타,검색,1000207,내부
4,검색,기타,1000207,내부
...,...,...,...,...
411626,기타,이탈,999960,외부
411627,고객센터,이탈,999973,외부
411631,기타,이탈,999980,외부
411639,기타,이탈,999994,외부


In [22]:
nunique = combined.groupby(['User Type', 'start', 'end'])['New IP'].nunique().reset_index()
count = combined.groupby(['User Type', 'start', 'end'])['New IP'].count().reset_index()

In [8]:
nunique

,User Type,start,end,New IP
0,내부,e-아카데미,검색,1
1,내부,e-아카데미,고객센터,10
2,내부,e-아카데미,기타,162
3,내부,e-아카데미,뉴스,46
4,내부,e-아카데미,매거진,353
...,...,...,...,...
312,외부,포토갤러리,이탈,156
313,외부,포토갤러리,인기통합정보,15
314,외부,포토갤러리,작품/조경수/자재,30
315,외부,포토갤러리,채용/인재,32


In [9]:
count

,User Type,start,end,New IP
0,내부,e-아카데미,검색,1
1,내부,e-아카데미,고객센터,11
2,내부,e-아카데미,기타,220
3,내부,e-아카데미,뉴스,55
4,내부,e-아카데미,매거진,1502
...,...,...,...,...
312,외부,포토갤러리,이탈,156
313,외부,포토갤러리,인기통합정보,17
314,외부,포토갤러리,작품/조경수/자재,63
315,외부,포토갤러리,채용/인재,40


In [23]:
inner_nunique = nunique.loc[nunique['User Type'] == '내부']
outer_nunique = nunique.loc[nunique['User Type'] == '외부']

inner_count = count.loc[count['User Type'] == '내부']
outer_count = count.loc[count['User Type'] == '외부']

In [24]:
#sum 
inner_nunique_sum = inner_nunique.groupby(['start'])['New IP'].sum().reset_index()
outer_nunique_sum = outer_nunique.groupby(['start'])['New IP'].sum().reset_index()
inner_count_sum = inner_count.groupby(['start'])['New IP'].sum().reset_index()
outer_count_sum = outer_count.groupby(['start'])['New IP'].sum().reset_index()

In [12]:
##inner_nunique_stats
inner_nunique_stats = pd.merge(inner_nunique, inner_nunique_sum, how='left', on='start')
inner_nunique_stats = pd.merge(inner_nunique_stats, outer_nunique_sum, how='left', on='start')
inner_nunique_stats = inner_nunique_stats.rename(columns={'New IP_x': '접속자 수', 'New IP_y': '내부 총합', 'New IP': '외부 총합'})
inner_nunique_stats['내외부 총합'] = inner_nunique_stats['내부 총합'] + inner_nunique_stats['외부 총합']
inner_nunique_stats['내부 비율'] = round((inner_nunique_stats['접속자 수'] / inner_nunique_stats['내부 총합']) * 100, 2)
inner_nunique_stats['전체 비율'] = round((inner_nunique_stats['접속자 수'] / inner_nunique_stats['내외부 총합']) * 100, 2)
inner_nunique_stats.to_csv("inner_nunique_stats.csv", index=False, encoding='cp949')

In [13]:
##inner_count_stats
inner_count_stats = pd.merge(inner_count, inner_count_sum, how='left', on='start')
inner_count_stats = pd.merge(inner_count_stats, outer_count_sum, how='left', on='start')
inner_count_stats = inner_count_stats.rename(columns={'New IP_x': '건수', 'New IP_y': '내부 총합', 'New IP': '외부 총합'})
inner_count_stats['내외부 총합'] = inner_count_stats['내부 총합'] + inner_count_stats['외부 총합']
inner_count_stats['내부 비율'] = round((inner_count_stats['건수'] / inner_count_stats['내부 총합']) * 100, 2)
inner_count_stats['전체 비율'] = round((inner_count_stats['건수'] / inner_count_stats['내외부 총합']) * 100, 2)
inner_count_stats.to_csv("inner_count_stats.csv", index=False, encoding='cp949')

In [14]:
##outer_nunique_stats
outer_nunique_stats = pd.merge(outer_nunique, outer_nunique_sum, how='left', on='start')
outer_nunique_stats = pd.merge(outer_nunique_stats, inner_nunique_sum, how='left', on='start')
outer_nunique_stats = outer_nunique_stats.rename(columns={'New IP_x': '접속자 수', 'New IP_y': '외부 총합', 'New IP': '내부 총합'})
outer_nunique_stats['내외부 총합'] = outer_nunique_stats['내부 총합'] + outer_nunique_stats['외부 총합']
outer_nunique_stats['외부 비율'] = round((outer_nunique_stats['접속자 수'] / outer_nunique_stats['외부 총합']) * 100, 2)
outer_nunique_stats['전체 비율'] = round((outer_nunique_stats['접속자 수'] / outer_nunique_stats['내외부 총합']) * 100, 2)
outer_nunique_stats.to_csv("outer_nunique_stats.csv", index=False, encoding='cp949')

In [25]:
##outer_count_stats
outer_count_stats = pd.merge(outer_count, outer_count_sum, how='left', on='start')
outer_count_stats = pd.merge(outer_count_stats, inner_count_sum, how='left', on='start')
outer_count_stats = outer_count_stats.rename(columns={'New IP_x': '건수', 'New IP_y': '외부 총합', 'New IP': '내부 총합'})
outer_count_stats['내외부 총합'] = outer_count_stats['내부 총합'] + outer_count_stats['외부 총합']
outer_count_stats['외부 비율'] = round((outer_count_stats['건수'] / outer_count_stats['외부 총합']) * 100, 2)
outer_count_stats['전체 비율'] = round((outer_count_stats['건수'] / outer_count_stats['내외부 총합']) * 100, 2)
outer_count_stats.to_csv("outer_count_stats.csv", index=False, encoding='cp949')

In [16]:
#Sankey Diagram generator

def draw_sankey(df):
    nodes = list(set(df['start']).union(set(df['end'])))
    links = []
    for _, row in df.iterrows():
        links.append({'source': nodes.index(row['start']), 'target': nodes.index(row['end']), 'value': row['New IP']})

    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad=15,
            thickness=20, 
            line=dict(color='black', width=0.5),
            label=[str(node) for node in nodes]
        ),
        link = dict(
            source = [link['source'] for link in links],
            target = [link['target'] for link in links], 
            value = [link['value'] for link in links]
        )
    )])

    fig.update_layout(width=500, height=500, font=dict(size=20))

    fig.show()

In [17]:
print(inner_nunique['start'].nunique())
print(inner_nunique['start'].unique())

13
['e-아카데미' '검색' '고객센터' '기타' '뉴스' '마이페이지' '매거진' '메인' '인기통합정보' '작품/조경수/자재'
 '채용/인재' '커뮤니티' '포토갤러리']


In [18]:
inner_nunique1 = inner_nunique[inner_nunique['start'].isin(['e-아카데미'])]
inner_nunique2 = inner_nunique[inner_nunique['start'].isin(['검색'])]
inner_nunique3 = inner_nunique[inner_nunique['start'].isin(['고객센터'])]
inner_nunique4 = inner_nunique[inner_nunique['start'].isin(['기타'])]
inner_nunique5 = inner_nunique[inner_nunique['start'].isin(['뉴스'])]
inner_nunique6 = inner_nunique[inner_nunique['start'].isin(['마이페이지'])]
inner_nunique7 = inner_nunique[inner_nunique['start'].isin(['매거진'])]
inner_nunique8 = inner_nunique[inner_nunique['start'].isin(['메인'])]
inner_nunique9 = inner_nunique[inner_nunique['start'].isin(['인기통화정보'])]
inner_nunique10 = inner_nunique[inner_nunique['start'].isin(['작품/조경수/자재'])]
inner_nunique11 = inner_nunique[inner_nunique['start'].isin(['채용/인재'])]
inner_nunique12 = inner_nunique[inner_nunique['start'].isin(['커뮤니티'])]
inner_nunique13 = inner_nunique[inner_nunique['start'].isin(['포토갤러리'])]

In [19]:
inner_count1 = inner_count[inner_count['start'].isin(['e-아카데미'])]
inner_count2 = inner_count[inner_count['start'].isin(['검색'])]
inner_count3 = inner_count[inner_count['start'].isin(['고객센터'])]
inner_count4 = inner_count[inner_count['start'].isin(['기타'])]
inner_count5 = inner_count[inner_count['start'].isin(['뉴스'])]
inner_count6 = inner_count[inner_count['start'].isin(['마이페이지'])]
inner_count7 = inner_count[inner_count['start'].isin(['매거진'])]
inner_count8 = inner_count[inner_count['start'].isin(['메인'])]
inner_count9 = inner_count[inner_count['start'].isin(['인기통화정보'])]
inner_count10 = inner_count[inner_count['start'].isin(['작품/조경수/자재'])]
inner_count11 = inner_count[inner_count['start'].isin(['채용/인재'])]
inner_count12 = inner_count[inner_count['start'].isin(['커뮤니티'])]
inner_count13 = inner_count[inner_count['start'].isin(['포토갤러리'])]


In [20]:
outer_nunique1 = outer_nunique[outer_nunique['start'].isin(['e-아카데미'])]
outer_nunique2 = outer_nunique[outer_nunique['start'].isin(['검색'])]
outer_nunique3 = outer_nunique[outer_nunique['start'].isin(['고객센터'])]
outer_nunique4 = outer_nunique[outer_nunique['start'].isin(['기타'])]
outer_nunique5 = outer_nunique[outer_nunique['start'].isin(['뉴스'])]
outer_nunique6 = outer_nunique[outer_nunique['start'].isin(['마이페이지'])]
outer_nunique7 = outer_nunique[outer_nunique['start'].isin(['매거진'])]
outer_nunique8 = outer_nunique[outer_nunique['start'].isin(['메인'])]
outer_nunique9 = outer_nunique[outer_nunique['start'].isin(['인기통화정보'])]
outer_nunique10 = outer_nunique[outer_nunique['start'].isin(['작품/조경수/자재'])]
outer_nunique11 = outer_nunique[outer_nunique['start'].isin(['채용/인재'])]
outer_nunique12 = outer_nunique[outer_nunique['start'].isin(['커뮤니티'])]
outer_nunique13 = outer_nunique[outer_nunique['start'].isin(['포토갤러리'])]

In [21]:
outer_count1 = outer_count[outer_count['start'].isin(['e-아카데미'])]
outer_count2 = outer_count[outer_count['start'].isin(['검색'])]
outer_count3 = outer_count[outer_count['start'].isin(['고객센터'])]
outer_count4 = outer_count[outer_count['start'].isin(['기타'])]
outer_count5 = outer_count[outer_count['start'].isin(['뉴스'])]
outer_count6 = outer_count[outer_count['start'].isin(['마이페이지'])]
outer_count7 = outer_count[outer_count['start'].isin(['매거진'])]
outer_count8 = outer_count[outer_count['start'].isin(['메인'])]
outer_count9 = outer_count[outer_count['start'].isin(['인기통화정보'])]
outer_count10 = outer_count[outer_count['start'].isin(['작품/조경수/자재'])]
outer_count11 = outer_count[outer_count['start'].isin(['채용/인재'])]
outer_count12 = outer_count[outer_count['start'].isin(['커뮤니티'])]
outer_count13 = outer_count[outer_count['start'].isin(['포토갤러리'])]

#### 내부 

In [22]:
draw_sankey(inner_nunique1)

In [23]:
draw_sankey(inner_count1)

In [24]:
draw_sankey(outer_nunique1)

In [25]:
draw_sankey(outer_count1)

In [26]:
draw_sankey(inner_nunique2)

In [27]:
draw_sankey(inner_count2)

In [28]:
draw_sankey(outer_nunique2)

In [29]:
draw_sankey(outer_count2)

In [64]:
draw_sankey(inner_nunique3)

In [65]:
draw_sankey(inner_count3)

In [66]:
draw_sankey(outer_nunique3)

In [67]:
draw_sankey(outer_count3)

In [30]:
draw_sankey(inner_nunique4)

In [31]:
draw_sankey(inner_count4)

In [32]:
draw_sankey(outer_nunique4)

In [33]:
draw_sankey(outer_count4)

In [34]:
draw_sankey(inner_nunique5)

In [35]:
draw_sankey(inner_count5)

In [36]:
draw_sankey(outer_nunique5)

In [37]:
draw_sankey(outer_count5)

In [38]:
draw_sankey(inner_nunique6)

In [39]:
draw_sankey(inner_count6)

In [40]:
draw_sankey(outer_nunique6)

In [41]:
draw_sankey(outer_count6)

In [42]:
draw_sankey(inner_nunique7)

In [43]:
draw_sankey(inner_count7)

In [44]:
draw_sankey(outer_nunique7)

In [45]:
draw_sankey(outer_count7)

In [46]:
draw_sankey(inner_nunique8)

In [47]:
draw_sankey(inner_count8)

In [48]:
draw_sankey(outer_nunique8)

In [49]:
draw_sankey(outer_count8)

In [52]:
draw_sankey(inner_nunique10)

In [53]:
draw_sankey(inner_count10)

In [54]:
draw_sankey(outer_nunique10)

In [55]:
draw_sankey(outer_count10)

In [56]:
draw_sankey(inner_nunique11)

In [57]:
draw_sankey(inner_count11)

In [58]:
draw_sankey(outer_nunique11)

In [59]:
draw_sankey(outer_count11)

In [60]:
draw_sankey(inner_nunique12)

In [61]:
draw_sankey(inner_count12)

In [62]:
draw_sankey(outer_nunique12)

In [63]:
draw_sankey(outer_count12)

In [64]:
draw_sankey(inner_nunique13)

In [65]:
draw_sankey(inner_count13)

In [66]:
draw_sankey(outer_nunique13)

In [67]:
draw_sankey(outer_count13)

#### 시작 경로 분석 

In [68]:
#'시작' 경로 추가
start_df = sankey_df.drop_duplicates(subset=['New IP'], keep='first')
start_df

,start,end,New IP,User Type
0,뉴스,매거진,1000201,내부
1,채용/인재,검색,1000207,내부
8,검색,기타,1000216,내부
18,검색,기타,1000226,내부
20,메인,뉴스,1000236,내부
...,...,...,...,...
411626,작품/조경수/자재,기타,999960,외부
411627,기타,고객센터,999973,외부
411628,기타,고객센터,999980,외부
411632,기타,고객센터,999994,외부


In [70]:
start_df.to_csv('start_df.csv', index=False, encoding='cp949')